# LLM Decisions -- If This, Else That

Some regulations have **exceptions**. The LLM classifies the situation, our code branches on the result.

In [ ]:
import os, json
from google import genai
from google.genai import types

os.environ["GOOGLE_API_KEY"] = "YOUR_KEY_HERE"
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

## The Regulation

**Headroom** (CTE SUA 2): the minimum ceiling height *depends on the space type*:

| Space type | Min height |
|---|---|
| General circulation | 2.20 m |
| Restricted use (private) | 2.10 m |
| Doorframe | 2.00 m |

Simple Python can't decide which rule applies -- the LLM reads the description and classifies.

In [ ]:
REGULATION = """
Minimum clear height: 2.20m in general circulation zones,
2.10m in restricted-use areas (private hallways inside apartments),
2.00m at doorframes.
"""

description = "Main corridor on floor 2 has a ceiling height of 2.15m"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"""
Regulation: {REGULATION}
Element: {description}

Classify the space and check compliance.
Return ONLY JSON: {{"space_type": "general"|"restricted"|"doorframe", "height_m": number, "required_m": number, "passed": bool}}
""",
)

text = response.text.strip().removeprefix("```json").removesuffix("```").strip()
data = json.loads(text)
print(data)

## Branch on the result

In [ ]:
if data["passed"]:
    print(f"PASS -- {data['space_type']} zone: {data['height_m']}m >= {data['required_m']}m")
elif data["space_type"] == "general":
    print(f"FAIL -- general zone needs 2.20m, got {data['height_m']}m")
elif data["space_type"] == "restricted":
    print(f"FAIL -- restricted zone needs 2.10m, got {data['height_m']}m")
else:
    print(f"FAIL -- doorframe needs 2.00m, got {data['height_m']}m")

## Try a different space

In [ ]:
# Change this and re-run!
description = "Private hallway inside apartment 3B has ceiling height 2.12m"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"""
Regulation: {REGULATION}
Element: {description}

Classify the space and check compliance.
Return ONLY JSON: {{"space_type": "general"|"restricted"|"doorframe", "height_m": number, "required_m": number, "passed": bool}}
""",
)

text = response.text.strip().removeprefix("```json").removesuffix("```").strip()
data = json.loads(text)

if data["passed"]:
    print(f"PASS -- {data['space_type']} zone: {data['height_m']}m >= {data['required_m']}m")
elif data["space_type"] == "general":
    print(f"FAIL -- general zone needs 2.20m, got {data['height_m']}m")
elif data["space_type"] == "restricted":
    print(f"FAIL -- restricted zone needs 2.10m, got {data['height_m']}m")
else:
    print(f"FAIL -- doorframe needs 2.00m, got {data['height_m']}m")

## Key Takeaways

1. The LLM **classifies** (general / restricted / doorframe)
2. Our code **branches** with plain `if / elif / else`
3. This handles regulations that simple `>=` checks can't
4. **Next:** PydanticAI automates the JSON parsing + adds tools